In [ ]:
#!pip install pmdarima
#!pip install fbprophet
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import time

# Load specific forecasting tools
from statsmodels.tsa.statespace.sarimax import SARIMAX

from statsmodels.graphics.tsaplots import plot_acf,plot_pacf # for determining (p,q) orders
from statsmodels.tsa.seasonal import seasonal_decompose      # for ETS Plots
from pmdarima import auto_arima                              # for determining ARIMA orders

# Load specific evaluation tools
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse

# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
from dateutil.relativedelta import relativedelta
from fbprophet import Prophet

from statsmodels.tsa.holtwinters import ExponentialSmoothing
import math

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import regularizers, optimizers
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline



In [ ]:
def model_creation(input_dim, h1, h2):
    model = Sequential()
    model.add(Dense(h1, input_dim=input_dim))
    #model.add(Dropout(0.1))
    model.add(Activation('relu'))
    #model.add(BatchNormalization())
    model.add(Dense(h2))
    model.add(Activation('relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(1))
    opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return (model)

#convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1, offset=0):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-offset):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back+offset])
    return np.array(dataX), np.array(dataY)

def nn_pred(train_nn):
    model = model_creation(12,6,6)
    nn_train = train_nn['y'].values
    X, y = create_dataset(nn_train, 12)
    model.fit(X, y, epochs=20,shuffle=True,verbose=0)
    predictions_nn=[]
    trainy = train['y'].values
    for i in range(3):
        
        nn_train = trainy[-12:].reshape(-1,12)
        pred = model.predict(nn_train)
        predictions_nn.append(pred.flatten()[0])
        trainy = np.append(trainy, round(pred.flatten()[0]) )
        
    predictions_nn = np.array(predictions_nn).round()
    return(predictions_nn )

def lstm_pred(train, testlen):
    n_input = 12
    n_features=1

    scaler = MinMaxScaler()
    scaler.fit(train)
    scaled_train = scaler.transform(train)
    #scaled_test = scaler.transform(test)

    generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)
    model = Sequential()
    model.add(LSTM(150, activation='relu', input_shape=(n_input, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit_generator(generator,epochs=25, verbose = 0)

    test_predictions = []
    first_eval_batch = scaled_train[-n_input:]
    current_batch = first_eval_batch.reshape((1, n_input, n_features))
    #print(current_batch)
    for i in range(testlen):
        #print(current_batch[0])
        current_pred = model.predict(current_batch)[0]
        test_predictions.append(current_pred) 
        # From 2nd dim of current batch, drop first value and add latest predition
        current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
    true_predictions = np.round(scaler.inverse_transform(test_predictions).flatten())
    return true_predictions



In [ ]:
df = pd.read_excel('.xlsx', sheet_name = 'FF -MFG Bookings 500')
print(df.shape)
df['PLID'] = df['PLID'].astype(str)
#df[df['PLID'].str.contains('4022058')]

In [ ]:
#df.groupby('Fiscal_Year_Month_Number').size()
df['ds'] = pd.to_datetime(df['Fiscal_Year_Month_Number'], format="%Y%m")
df.describe(include='all')

In [ ]:
df['ds'].value_counts()

In [ ]:
#df['ds'] = pd.to_datetime(df['Fiscal Year Quarter Number'], format="%Y%q")

In [ ]:
df.head(2)

In [ ]:
#Getting plids with missing months
# for idx, plid in enumerate(df_fc['PLID'].values):   
#     df1 = df [ (df['PLID'] ==  plid ) ]
#     #print(f'\n*****{idx} - PLID: {plid} Shape: {df1.shape}*****')
#     ds_array = np.arange ( df1['ds'].min(), df1['ds'].max()+ 
#                       relativedelta(months=1), 1 , dtype='datetime64[M]') 
#     #print(ds_array)
#     df1.sort_values(by = 'ds', inplace=True)
#     if len(ds_array) == len(df1): 
#         df1['ds'] = ds_array
#     else:
#         #print (f"'{plid}',")
#         print (plid,len(ds_array),len(df1)  )

In [ ]:
df_fc = pd.read_excel('./PRO/PRO Version - 500 PLIDs .xlsx', sheet_name = 'FantasyForecast PRO')
df_fc['PLID'] = df_fc['PLID'].astype(str)
print(df_fc.shape)
df_fc.rename(columns ={'PLID (DP)': 'PLID'}, inplace = True)
display(df_fc.head() )
df_fc['model_num'] =  None
df_fc['rmse'] = None
df_fc['rmse_pct'] = None
df_fc['all_mean'] = None
df_fc['test_mean'] = None
df_fc['train_mean']= None
df_fc['latest_6']= None
df_fc['predicted']= None
df_fc['accuracy']= None

In [ ]:
df_fc['PLID'] = df_fc['PLID'].astype(str)

In [ ]:
import time
#df_fc = pd.read_csv('forecast_q2_acc_v1.csv')
for idx, plid in enumerate(df_fc['PLID'].values[442:]) :

    df1 = df [ (df['PLID'] ==  plid ) ]
    print(f'\n*****{idx} - PLID: {plid} Shape: {df1.shape} {time.asctime()}*****')
    
    #If rows are less than 4 just put average                       
    if df1.shape[0] <= 6:
        q_forecast = round(df1['MFG_REV_BKG_QTY'].mean()*3/df1.shape[0])
        model_num=99     
        cols = ['Q2 FY2021 Forecast (Units)', 'model_num']              
        df_fc.loc[df_fc['PLID']==plid, cols] = [q_forecast, model_num] 
        continue
    
    df1.sort_values(by = 'ds', inplace=True)

    ds_array = np.arange ( df1['ds'].min(), df1['ds'].max()+ 
                      relativedelta(months=1), 1 , dtype='datetime64[M]') 
    
    #Extraploating missing values in case dates are missing
    if len(ds_array) != len(df1):
        for dt in ds_array:
            if len( df1[df1['ds']==dt]) != 0:
                y = df1[df1['ds']==dt]['MFG_REV_BKG_QTY'].values[0]
               
            else:
                df1 = df1.append({'PLID': plid, 'ds':dt,'MFG_REV_BKG_QTY': y }, ignore_index=True)
            df1.sort_values(by = 'ds', inplace=True)
    
    df1 = df1[['ds', 'MFG_REV_BKG_QTY'] ]
    df1.columns = ['ds', 'y']
 
    #df1['ds'] = pd.to_datetime(df1['ds'])
    train = df1.iloc[:-3]
    test = df1.iloc[-3:]
    test_mean = round (test['y'].mean())
    train_mean = round (train['y'].mean())
    all_mean = round (df1['y'].mean())
    print(f'All Mean: {all_mean} Train Mean: {train_mean} Test Mean: {test_mean}')
    #print(train)
    
#   Bias = (Forecast-Actual)/Actual
#   Accuracy = 1 - Abs(Bias)
  
    
    
    # Prohet model
    try:
        m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality= False)
        m.fit(train)
        future = m.make_future_dataframe(len(test), freq='MS')
        forecast = m.predict(future)

        predictions = forecast.iloc[-1*len(test):]['yhat']
        error_p_a = round(rmse(predictions,test['y']),2)
        bias = (predictions.sum() - test['y'].values.sum())/test['y'].values.sum()
        acc_p_a = round( 1- abs(bias ),4)
    except:
        print ('Prophet Exception')
        error_p_a = math.nan
        acc_p_a = math.nan
        
    
    # Prohet model Multiplicative
    try:
        m = Prophet(yearly_seasonality=True , weekly_seasonality=False, daily_seasonality= False, 
                    seasonality_mode='multiplicative')
        m.fit(train)
        future = m.make_future_dataframe(len(test), freq='MS')

        forecast = m.predict(future)

        predictions = forecast.iloc[-1*len(test):]['yhat']
        error_p_m = round(rmse(predictions,test['y']),2)
        bias = (predictions.sum() - test['y'].values.sum())/test['y'].values.sum()
        acc_p_m = round( 1- abs(bias ),4)
    except:
        print ('Prophet Exception')
        error_p_m = math.nan
        acc_p_m = math.nan
    
    print(f'Prophet Error - Add:{error_p_a} Mul:{error_p_m}, Prophet Accuracy - Add:{acc_p_a} Mul:{acc_p_m} ' ) 
   
    
    
    
  
    #ARIMA
    df1.set_index('ds', inplace=True)
    df1.index.freq = 'MS'
    train = df1.iloc[:-3]
    test = df1.iloc[-3:]
  

    
    if len(df1)> 39:
        s=True
        m=12
    else:
        s = False
        m=1
    
   
    #print(df1.info(), df1.index, df1.shape, df1)
    try:
        model = auto_arima(df1['y'], seasonal=s, m=m, trend='c')
        model = SARIMAX( train['y'], order = model.order, seasonal_order= model.seasonal_order)
        results = model.fit()
        predictions = results.predict ( start = len(train), end=len(train)+len(test)-1, dynamic=False, 
                                 typ='levels').rename('Prediction')
        error_a_c1 = round(rmse(predictions,test['y']),2)
        bias = (predictions.sum() - test['y'].values.sum())/test['y'].values.sum()
        acc_a_c1 = round( 1- abs(bias ),4)
    except:
        print ('Exception')
        error_a_c1 = math.nan
        acc_a_c1 = math.nan
        
  
    try:
        model = auto_arima(df1['y'], seasonal=s, m=m, trend='t')
        model = SARIMAX( train['y'], order = model.order, seasonal_order= model.seasonal_order)
        results = model.fit()
        predictions = results.predict ( start = len(train), end=len(train)+len(test)-1, dynamic=False, 
                                       typ='levels').rename('Prediction')
        error_a_t1 = round(rmse(predictions,test['y']),2)
        bias = (predictions.sum() - test['y'].values.sum())/test['y'].values.sum()
        acc_a_t1 = round( 1- abs(bias ),4)
        
    except:
        print ('Exception')
        error_a_t1 = math.nan
        acc_a_t1 = math.nan
   
    try:
        model = auto_arima(df1['y'], seasonal=True, m=m, trend='ct')
        model = SARIMAX( train['y'], order = model.order, seasonal_order= model.seasonal_order)
        results = model.fit()
        predictions = results.predict ( start = len(train), end=len(train)+len(test)-1, dynamic=False, 
                                       typ='levels').rename('Prediction')
        error_a_ct1 = round(rmse(predictions,test['y']),2)
        bias = (predictions.sum() - test['y'].values.sum())/test['y'].values.sum()
        acc_a_ct1 = round( 1- abs(bias ),4)
    except:
        print ('Exception')
        error_a_ct1 = math.nan
        acc_a_ct1 = math.nan
    
    print(f'ARIMA Error - c:{error_a_c1} t:{error_a_t1}  ct:{error_a_ct1}, ARIMA Accuracy - c:{acc_a_c1} t:{acc_a_t1}  ct:{acc_a_ct1}  ' ) 
  
    
    #Holt winter
#     if len(df1)>= 27:
#         m1=12
#     else:
#         m1=3
    
#     try:
#         fitted_model = ExponentialSmoothing(train['y'],trend='add',seasonal='add',seasonal_periods=m1).fit()
#         predictions = fitted_model.forecast(len(test)).rename('HW Forecast')
#         error_hwaa = round(rmse(predictions,test['y']),2)
#         bias = (predictions.sum() - test['y'].values.sum())/test['y'].values.sum()
#         acc_hwaa = round( 1- abs(bias ),4)
#     except:
#         print ('Exception HWaa')
#         error_hwaa = math.nan
#         acc_hwaa = math.nan
    
#     try:
#         fitted_model = ExponentialSmoothing(train['y'],trend='add',seasonal='mul',seasonal_periods=m1).fit()
#         predictions = fitted_model.forecast(len(test)).rename('HW Forecast')
#         error_hwam = round(rmse(predictions,test['y']),2)
#         bias = (predictions.sum() - test['y'].values.sum())/test['y'].values.sum()
#         acc_hwam = round( 1- abs(bias ),4)
#     except:
#         print ('Exception in HWam')
#         error_hwam = math.nan
#         acc_hwam = math.nan
    
#     try:
#         fitted_model = ExponentialSmoothing(train['y'],trend='mul',seasonal='add',seasonal_periods=m1).fit()
#         predictions = fitted_model.forecast(len(test)).rename('HW Forecast')
#         error_hwma = round(rmse(predictions,test['y']),2)
#         bias = (predictions.sum() - test['y'].values.sum())/test['y'].values.sum()
#         acc_hwma = round( 1- abs(bias ),4)
#     except:
#         print ('Exception in HWma')
#         error_hwma = math.nan
#         acc_hwma = math.nan
    
#     try:
#         fitted_model = ExponentialSmoothing(train['y'],trend='mul',seasonal='mul',seasonal_periods=m1).fit()
#         predictions = fitted_model.forecast(len(test)).rename('HW Forecast')
#         error_hwmm = round(rmse(predictions,test['y']),2)
#         bias = (predictions.sum() - test['y'].values.sum())/test['y'].values.sum()
#         acc_hwmm = round( 1- abs(bias ),4)
#     except:
#         print ('Exception HWmm')
#         error_hwmm = math.nan
#         acc_hwmm = math.nan
        
#     print(f'HW Error - aa:{error_hwaa} am:{error_hwam} ma:{error_hwma} mm:{error_hwmm}, HW Accuracy - aa:{acc_hwaa} am:{acc_hwam} ma:{acc_hwma} mm:{acc_hwmm}  ' ) 


    #Neural Network
#     try:
#         predictions = lstm_pred(train, len(test) )
#         error_nn = round( np.sqrt(mean_squared_error(predictions,test['y'].values)), 2)
#         bias = (predictions.sum() - test['y'].values.sum())/test['y'].values.sum()
#         acc_nn = round( 1- abs(bias ),4)
        
#         error_nn = math.inf
#         acc_nn = - math.inf
#     except:
#         print ('Exception in lstm model')
#         error_nn = math.nan
#         acc_nn =  math.nan
#     print(f'LSTM Error: {error_nn}, LSTM Accuracy: {acc_nn}')

    error_hwaa = math.inf
    acc_hwaa = - math.inf
    error_hwam = math.inf
    acc_hwam = - math.inf
    error_hwma = math.inf
    acc_hwma = - math.inf
    error_hwmm = math.inf
    acc_hwmm = - math.inf    
    error_nn = math.inf
    acc_nn = - math.inf


    error_list = [error_p_a, error_p_m,error_a_c1, error_a_t1, error_a_ct1 , error_hwaa, error_hwam, 
                  error_hwma, error_hwmm, error_nn]
    error_list = np.nan_to_num(error_list, copy=True, nan=math.inf)
    
    acc_list = [acc_p_a, acc_p_m,acc_a_c1, acc_a_t1, acc_a_ct1 , acc_hwaa, acc_hwam, 
                  acc_hwma, acc_hwmm, acc_nn]
    acc_list = np.nan_to_num(acc_list, copy=True, nan= -math.inf)
    
    model_num = np.argmin(error_list)
    #model_num = np.argmax(acc_list)
    
    
    
    #print(model_num)

    rmse_final = error_list[model_num]
    rmse_pct = round(error_list[model_num]*100/test_mean,2)
    acc_final = acc_list[model_num]
    latest_6 = df1['y'].values[-6:]
    if model_num == 0:
        if df1.shape[1] == 1:
            df1.reset_index(inplace=True)
        m = Prophet()
        m.fit(df1)
        future = m.make_future_dataframe(3, freq='MS')
        forecast = m.predict(future)
        predictions = forecast.iloc[-3:]['yhat'].values
    elif model_num == 1:
        if df1.shape[1] == 1:
            df1.reset_index(inplace=True)
        m = Prophet(seasonality_mode='multiplicative')
        m.fit(df1)
        future = m.make_future_dataframe(3, freq='MS')
        forecast = m.predict(future)
        predictions = forecast.iloc[-3:]['yhat'].values
    elif model_num == 2:
        model = auto_arima(df1['y'], seasonal=s, m=m, trend='c')
        model = SARIMAX( df1['y'], order = model.order, seasonal_order= model.seasonal_order)
        results = model.fit()
        predictions = results.predict ( start = len(df1), end=len(df1)+2, dynamic=False, 
                                   typ='levels').values
    elif model_num == 3:
        model = auto_arima(df1['y'], seasonal=s, m=m, trend='t')
        model = SARIMAX( df1['y'], order = model.order, seasonal_order= model.seasonal_order)
        results = model.fit()
        predictions = results.predict ( start = len(df1), end=len(df1)+2, dynamic=False, 
                                   typ='levels').values
    elif model_num == 4:
        model = auto_arima(df1['y'], seasonal=s, m=m, trend='ct')
        model = SARIMAX( df1['y'], order = model.order, seasonal_order= model.seasonal_order)
        results = model.fit()
        predictions = results.predict ( start = len(df1), end=len(df1)+2, dynamic=False, 
                                   typ='levels').values
    elif model_num == 5:
        fitted_model = ExponentialSmoothing(df1['y'],trend='add',seasonal='add',seasonal_periods=m1).fit()
        predictions = fitted_model.forecast(3).values
    elif model_num == 6:
        fitted_model = ExponentialSmoothing(df1['y'],trend='add',seasonal='mul',seasonal_periods=m1).fit()
        predictions = fitted_model.forecast(3).values
    elif model_num == 7:
        fitted_model = ExponentialSmoothing(df1['y'],trend='mul',seasonal='add',seasonal_periods=m1).fit()
        predictions = fitted_model.forecast(3).values
    elif model_num == 8:
        fitted_model = ExponentialSmoothing(df1['y'],trend='mul',seasonal='mul',seasonal_periods=m1).fit()
        predictions = fitted_model.forecast(3).values
    elif model_num == 9:
        predictions = lstm_pred(df1,3)
    else:
        print('There is an issue')
    
    print (f'Model Num:{model_num} RMSE={rmse_final} RMSE%={rmse_pct} Accuracy={acc_final} Latest 6={latest_6} Predictions={predictions.round()}')
    
#     plt.figure(figsize=(10,1))
#     plt.plot(range(9), np.concatenate([latest_6,predictions]), marker='o' )
    
    q_forecast = round(predictions.sum())
    #print(plid, q_forecast, rmse_final,rmse_pct, test_mean,train_mean, model_num)
    
    cols = ['Q2 FY2021 Forecast (Units)', 'model_num', 'rmse', 'rmse_pct', 'accuracy','all_mean','test_mean', \
            'train_mean', 'latest_6', 'predicted' ]
    
    df_fc.loc[df_fc['PLID']==plid, cols] = \
         [q_forecast, model_num, rmse_final,rmse_pct,acc_final, all_mean, test_mean,train_mean, str(latest_6),str(predictions.round()) ] 
    
    df_fc.to_csv('forecast_q2_acc_v1.csv', index=False)
#     print(df1.shape, predictions.shape)   
#     title='Booking Forecast Model #' + str(model_num) 
#     ylabel='Monthly Bookings'
#     xlabel=''
#     ax = df1.plot(legend=True,figsize=(12,4),title=title)
#     predictions.plot(legend=True)
#     ax.autoscale(axis='x',tight=True)
#     ax.set(xlabel=xlabel, ylabel=ylabel);
#     time.sleep(0.1)
#     plt.pause(0.0001)  
        
    

              
    

In [ ]:
df[df['PLID']=='87977']

In [ ]:
df_fc.shape

In [ ]:
#df_fc.to_csv('forecast_q2_acc_v1.csv', index=False)

In [ ]:
df_fc.groupby('model_num').size()

In [ ]:
#df_fc = pd.read_csv('forecast_q2_acc_v1.csv')
df_fc.groupby('model_num').mean()

In [ ]:
df_fc[df_fc['model_num'].notnull()].tail(3)

In [ ]:
print(df_fc[df_fc['model_num'].notnull()].shape )
df_fc[df_fc['model_num'].notnull()].tail(3)

In [ ]:
#df_fc[df_fc['model_num']== 2]

In [ ]:
df1 =  pd.read_csv('forecast_q2_acc_v1.csv')
df1.shape

In [ ]:
df1['Forecast'] = df1['Q2 FY2021 Forecast (Units)']

In [ ]:
df1.loc[ (df1['accuracy'] < .5) | (df1['accuracy'] == -math.inf)  , 'Forecast'] = df1['test_mean']*3

In [ ]:
df1.loc[df1['accuracy'] < .5]

In [ ]:
df2 = pd.merge ( df , df1[df1.model_num == 99]['PLID'], on ='PLID', how ='inner')

In [ ]:
df2.sort_values(by = ['PLID','ds' ], inplace = True)

In [ ]:
df2.shape

In [ ]:
df3 = df2.groupby("PLID")["MFG_REV_BKG_QTY"].apply(lambda x: round(x.iloc[-3:].sum()) )

In [ ]:
df3 = df3.reset_index()

In [ ]:
df3.rename({'MFG_REV_BKG_QTY': 'Forecast1'}, axis =1, inplace=True)

In [ ]:
df3

In [ ]:
df1= pd.merge(df1, df3, on = 'PLID', how='left' )

In [ ]:
df1.loc[df1.model_num == 99, 'Forecast'] = df1['Forecast1']

In [ ]:
df1.loc[df1.model_num == 99]

In [ ]:
df1.to_csv('forecast_q2_acc_v1_modified.csv', index = 'False')

In [ ]:
df.info()

In [ ]:
df.memory_usage(deep=True).sum()


In [ ]:
df = pd.read_excel ('forecast_q2_acc_v1_modified_subm.xlsx')
df.shape

In [ ]:
df1 = pd.read_excel ('Q2FY21 - 3 Cycle Forecast (STAT,CDO,BASE)_avg.xlsx')
df1.shape

In [ ]:
df1 = df1[['PLID (DP)', 'Avg']]

In [ ]:
df1 = df1.rename({'PLID (DP)': 'PLID'}, axis = 1)
df1

In [ ]:
df1['Avg'] = df1['Avg'].apply(round)

In [ ]:
df = pd.merge(df, df1, on ='PLID', how='left')

In [ ]:
df['Forecast_new'] = df['Forecast']

In [ ]:
df.loc[(df['accuracy']<= .7) | (df['model_num']==99), 'Forecast_new'] = df['Avg']

In [ ]:
df.head()

In [ ]:
#df['Forecast_new'] = df['Forecast_new'].astype('int64')

In [ ]:
df.to_csv('q2_new_forecast_subm.csv')